In [250]:
import math
import numpy as np
np.set_printoptions(suppress=True)

import pandas as pd

X_filename = "x.csv"
Y_filename = "y.csv"

In [215]:
def distance(x, y, alpha):
    dist = 0
    for i in range(len(x)):
        dist += (alpha[i] * (x[i] - y[i]))

    return abs(dist)

def similarity(dist, beta):
    return math.exp(-beta * dist)

In [216]:
def get_exempler_set(dataframe, category_R):

    category_dataframe = dataframe[dataframe.label == category_R]
    return category_dataframe[["weight", 'height']]


In [223]:
def get_count(x, exemplar_set):
    values_df = exemplar_set.value_counts().reset_index()
    x_weight, x_height = x.weight, x.height
    row = values_df.loc[(values_df.weight == x_weight) & (values_df.height == x_height)]
    return row.iloc[0][0]

def exemplar_vote(y, dataframe, category_R, gamma_R):
    exemplar_set = get_exempler_set(dataframe, category_R)
    # print(exemplar_set)
    prob_R_y = 0 
    for x in exemplar_set.iterrows():
        x_ = x[1]
        N_R_x = get_count(x_, exemplar_set)
        distance_x_y = distance(x_, y, alpha= [1,1])
        similarity_x_y = similarity(distance_x_y, beta=1)
        prob_R_y += N_R_x * similarity_x_y
    
    prob_R_y *= gamma_R 

    return prob_R_y
    
def get_probablity(y, dataframe,categories = [1, 2, 3]):
    prob = []
    prob.append(exemplar_vote(y, dataframe, 1, 1))
    prob.append(exemplar_vote(y, dataframe, 2, 1))
    prob.append(exemplar_vote(y, dataframe, 3, 1))

    return prob/sum(prob)

In [254]:
X_df = pd.read_csv(X_filename, names=['weight', 'height', 'label'])
Y_df = pd.read_csv(Y_filename, names=['weight', 'height',])

In [255]:
X_df.value_counts()

weight  height  label
67      67      2        2
65      73      2        2
64      63      2        2
        67      2        2
        69      2        2
                        ..
65      67      2        1
66      62      2        1
        65      2        1
        66      2        1
85      68      3        1
Length: 64, dtype: int64

In [226]:
Y_df.head()

,weight,height
0,74,67
1,69,63
2,92,81
3,64,61
4,66,84


In [247]:
# prob = get_probablity(Y_df.iloc[6], X_df)


0.9838621969282584

In [248]:
probablity = []
for i in range(len(Y_df)):
    probablity.append(get_probablity(Y_df.iloc[i], X_df))

In [251]:
probablity

[array([0.        , 0.90465068, 0.09534932]),
 array([0.00000004, 0.9999966 , 0.00000335]),
 array([0.        , 0.00000113, 0.99999887]),
 array([0.00037445, 0.99962553, 0.00000002]),
 array([0.        , 0.00010416, 0.99989584]),
 array([0.        , 0.07866294, 0.92133706]),
 array([0.9838622, 0.0161378, 0.       ]),
 array([0.        , 0.97291818, 0.02708182]),
 array([0.00000001, 0.99997427, 0.00002572]),
 array([1., 0., 0.])]